In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from joblib import dump, load

In [3]:
# folders with different resutls\
results_with_no_FS = './Results'
results_with_no_CORRANA = './Results_NOCORRANA'

subdirs = {
    'data':'INITIAL_SPLIT',
    'CORR':'CORR_ANA',
    'FS':'FS',
    'ML':'ML'
}

print(os.listdir(os.path.join(results_with_no_CORRANA,subdirs['CORR'])))

[]


In [9]:
# Make sure that both techinuqes have utilized same subjects
subjects_no_corr = os.listdir(os.path.join(results_with_no_CORRANA, subdirs['data']))
subjects_no_FS = os.listdir(os.path.join(results_with_no_FS, subdirs['data']))

print(subjects_no_corr, subjects_no_FS)

df_data_no_corr_left = pd.read_csv(os.path.join(results_with_no_CORRANA, subdirs['data'],'left_train_modifiedMedPIQR.csv')
                                  , index_col=0)
df_data_no_corr_right = pd.read_csv(os.path.join(results_with_no_CORRANA, subdirs['data'],'right_train_modifiedMedPIQR.csv')
                                  , index_col=0)

df_data_no_FS_left = pd.read_csv(os.path.join(results_with_no_FS, subdirs['data'],'left_train_modifiedMedPIQR.csv')
                                  , index_col=0)
df_data_no_FS_right = pd.read_csv(os.path.join(results_with_no_FS, subdirs['data'],'right_train_modifiedMedPIQR.csv')
                                  , index_col=0)

is_left_the_same = df_data_no_corr_left.index == df_data_no_FS_left.index
is_right_the_same = df_data_no_corr_right.index == df_data_no_FS_right.index
for i in is_left_the_same:
    if not i:
        print('LEFT IS FALSE')
        
for i in is_right_the_same:
    if not i:
        print('RIGHT IS FALSE')
        
"""
CONCLUSION:
Both techniques used the same subjects
"""

['left_train_modifiedMedPIQR.csv', 'right_train_modifiedMedPIQR.csv'] ['left_test.csv', 'left_test_modifiedMedPIQR.csv', 'left_train.csv', 'left_train_modifiedMedPIQR.csv', 'right_test.csv', 'right_test_modifiedMedPIQR.csv', 'right_train.csv', 'right_train_modifiedMedPIQR.csv']


## Given:
Features selected by <b>Correlation analysis at threshold 0.5, 0.6, 0.7, and 0.85</b>, <b>FS+RF</b>, and <b>FS+SVM</b>.
## Questions:
1. Which technique has the most reduction ration?
2. Are techniques selecting the same features?

In [35]:
# Change left and right names to keep only brain region and morphology
# without the added l and r
def remove_l_r_fromname(list_of_names):
    updated_names = []
    for name in list_of_names:
        nl = name.split('_')
        lrpos = nl[1]
        lrpos = lrpos[1:]
        nl[1] = lrpos
        updated_names.append('_'.join(nl))
    return updated_names


In [36]:
def find_matchings(f_LR, f_RR):
    updated_f_LR = remove_l_r_fromname(f_LR)
    updated_f_RR = remove_l_r_fromname(f_RR)
    common_featsLR = []
    common_featsRR = []
    only_LR_feats = []
    only_RR_feats = []
    for feat_LR in updated_f_LR:
        if feat_LR in updated_f_RR:
            common_featsLR.append(feat_LR)
        else:
            only_LR_feats.append(feat_LR)

    for feat_RR in updated_f_RR:
        if feat_RR in updated_f_LR:
            common_featsRR.append(feat_RR)
        else:
            only_RR_feats.append(feat_RR)
    return common_featsLR, common_featsRR, only_LR_feats, only_RR_feats

In [55]:
# Remove labels from list
def removeLabels(feat_list):
    if 'labels' in feat_list:
        try:
            feat_list.remove('labels')
        except:
            feat_list = feat_list.drop('labels')
            
    return feat_list

In [60]:
# Load correlation dataframes at different thresholds
df_corr_50_left = pd.read_csv(os.path.join(results_with_no_FS, subdirs['CORR'], 'left_train_noColliniarity_modifiedMedPIQR_50.csv'),
                         index_col=0)
df_corr_50_right = pd.read_csv(os.path.join(results_with_no_FS, subdirs['CORR'], 'right_train_noColliniarity_modifiedMedPIQR_50.csv'),
                         index_col=0)
left_feats_50 = removeLabels(df_corr_50_left.columns.to_list())
right_feats_50 = removeLabels(df_corr_50_right.columns.to_list())
print(len(left_feats_50), len(right_feats_50))


df_corr_60_left = pd.read_csv(os.path.join(results_with_no_FS, subdirs['CORR'], 'left_train_noColliniarity_modifiedMedPIQR_60.csv'),
                         index_col=0)
df_corr_60_right = pd.read_csv(os.path.join(results_with_no_FS, subdirs['CORR'], 'right_train_noColliniarity_modifiedMedPIQR_60.csv'),
                         index_col=0)
left_feats_60 = removeLabels(df_corr_60_left.columns.to_list())
right_feats_60 = removeLabels(df_corr_60_right.columns.to_list())


df_corr_70_left = pd.read_csv(os.path.join(results_with_no_FS, subdirs['CORR'], 'left_train_noColliniarity_modifiedMedPIQR_70.csv'),
                         index_col=0)
df_corr_70_right = pd.read_csv(os.path.join(results_with_no_FS, subdirs['CORR'], 'right_train_noColliniarity_modifiedMedPIQR_70.csv'),
                         index_col=0)
left_feats_70 = removeLabels(df_corr_70_left.columns.to_list())
right_feats_70 = removeLabels(df_corr_70_right.columns.to_list())


df_corr_85_left = pd.read_csv(os.path.join(results_with_no_FS, subdirs['CORR'], 'left_train_noColliniarity_modifiedMedPIQR_85.csv'),
                         index_col=0)
df_corr_85_right = pd.read_csv(os.path.join(results_with_no_FS, subdirs['CORR'], 'right_train_noColliniarity_modifiedMedPIQR_85.csv'),
                         index_col=0)
left_feats_85 = df_corr_85_left.columns.to_list()
right_feats_85 = df_corr_85_right.columns.to_list()

99 99


In [13]:
# Load RFECV objects
left_obj_rf = load(os.path.join(results_with_no_CORRANA, subdirs['FS'],'left_rf_modifiedMedPIQR_nocorr.joblib'))
left_obj_svm = load(os.path.join(results_with_no_CORRANA, subdirs['FS'],'left_svc_modifiedMedPIQR_nocorr.joblib'))
right_obj_rf = load(os.path.join(results_with_no_CORRANA, subdirs['FS'],'right_rf_modifiedMedPIQR_nocorr.joblib'))
right_obj_svm = load(os.path.join(results_with_no_CORRANA, subdirs['FS'],'right_svc_modifiedMedPIQR_nocorr.joblib'))


In [44]:
orig_col_names_left = df_data_no_corr_left.columns
orig_col_names_right = df_data_no_corr_right.columns
if 'labels' in orig_col_names_left:
    orig_col_names_left = orig_col_names_left.drop('labels')
    
if 'labels' in orig_col_names_right:
    orig_col_names_right = orig_col_names_right.drop('labels')
    

In [45]:
selected_feats_left_rf = orig_col_names_left[np.where(left_obj_rf.ranking_==1)[0]]
selected_feats_left_svm = orig_col_names_left[np.where(left_obj_svm.ranking_==1)[0]]
selected_feats_right_rf = orig_col_names_right[np.where(right_obj_rf.ranking_==1)[0]]
selected_feats_right_svm = orig_col_names_right[np.where(right_obj_svm.ranking_==1)[0]]

In [46]:
print('RF+left+RFECV: ', len(selected_feats_left_rf), selected_feats_left_rf)
print('SVM+left+RFECV: ', len(selected_feats_left_svm), selected_feats_left_svm)
print('RF+RIGHT+RFECV: ', len(selected_feats_right_rf), selected_feats_right_rf)
print('SVM+RIGHT+RFECV: ', len(selected_feats_right_svm), selected_feats_right_svm)

RF+left+RFECV:  6 Index(['thick_llateraloccipital_medMIQR', 'thick_lsuperiortemporal_medPIQR',
       'area_lbankssts_medMIQR', 'area_lpericalcarine_medPIQR',
       'area_lsupramarginal_medPIQR', 'vol_lbankssts_medMIQR'],
      dtype='object')
SVM+left+RFECV:  5 Index(['thick_linferiorparietal_medPIQR', 'thick_lsuperiorparietal_medPIQR',
       'vol_linferiorparietal_medPIQR', 'vol_llateraloccipital_medPIQR',
       'vol_lsuperiortemporal_medPIQR'],
      dtype='object')
RF+RIGHT+RFECV:  6 Index(['thick_rsuperiorparietal_medPIQR', 'thick_rsuperiortemporal_medPIQR',
       'area_rbankssts_medMIQR', 'area_rpericalcarine_medPIQR',
       'area_rsupramarginal_medPIQR', 'vol_rmedialorbitofrontal_medPIQR'],
      dtype='object')
SVM+RIGHT+RFECV:  5 Index(['thick_rinferiorparietal_medPIQR', 'thick_rsuperiorparietal_medPIQR',
       'vol_rinferiorparietal_medPIQR', 'vol_rlateraloccipital_medPIQR',
       'vol_rsuperiortemporal_medPIQR'],
      dtype='object')


In [47]:
# Difference betwee features selected by SVM and RF for left hemisphere
common_featsLR, common_featsRR, only_LR_feats, only_RR_feats = find_matchings(selected_feats_left_rf,
                                                                             selected_feats_left_svm)
common_featsLR, common_featsRR, only_LR_feats, only_RR_feats

([],
 [],
 ['thick_lateraloccipital_medMIQR',
  'thick_superiortemporal_medPIQR',
  'area_bankssts_medMIQR',
  'area_pericalcarine_medPIQR',
  'area_supramarginal_medPIQR',
  'vol_bankssts_medMIQR'],
 ['thick_inferiorparietal_medPIQR',
  'thick_superiorparietal_medPIQR',
  'vol_inferiorparietal_medPIQR',
  'vol_lateraloccipital_medPIQR',
  'vol_superiortemporal_medPIQR'])

In [48]:
# Difference betwee features selected by SVM and RF for Right hemisphere
common_featsLR, common_featsRR, only_LR_feats, only_RR_feats = find_matchings(selected_feats_right_rf,
                                                                             selected_feats_right_svm)
common_featsLR, common_featsRR, only_LR_feats, only_RR_feats

(['thick_superiorparietal_medPIQR'],
 ['thick_superiorparietal_medPIQR'],
 ['thick_superiortemporal_medPIQR',
  'area_bankssts_medMIQR',
  'area_pericalcarine_medPIQR',
  'area_supramarginal_medPIQR',
  'vol_medialorbitofrontal_medPIQR'],
 ['thick_inferiorparietal_medPIQR',
  'vol_inferiorparietal_medPIQR',
  'vol_lateraloccipital_medPIQR',
  'vol_superiortemporal_medPIQR'])

In [49]:
# Difference betwee features selected by RF for both hemisphere
common_featsLR, common_featsRR, only_LR_feats, only_RR_feats = find_matchings(selected_feats_right_rf,
                                                                             selected_feats_left_rf)
common_featsLR, common_featsRR, only_LR_feats, only_RR_feats

(['thick_superiortemporal_medPIQR',
  'area_bankssts_medMIQR',
  'area_pericalcarine_medPIQR',
  'area_supramarginal_medPIQR'],
 ['thick_superiortemporal_medPIQR',
  'area_bankssts_medMIQR',
  'area_pericalcarine_medPIQR',
  'area_supramarginal_medPIQR'],
 ['thick_superiorparietal_medPIQR', 'vol_medialorbitofrontal_medPIQR'],
 ['thick_lateraloccipital_medMIQR', 'vol_bankssts_medMIQR'])

In [50]:
# Difference betwee features selected by SVM for both hemisphere
common_featsLR, common_featsRR, only_LR_feats, only_RR_feats = find_matchings(selected_feats_right_svm,
                                                                             selected_feats_left_svm)
common_featsLR, common_featsRR, only_LR_feats, only_RR_feats

(['thick_inferiorparietal_medPIQR',
  'thick_superiorparietal_medPIQR',
  'vol_inferiorparietal_medPIQR',
  'vol_lateraloccipital_medPIQR',
  'vol_superiortemporal_medPIQR'],
 ['thick_inferiorparietal_medPIQR',
  'thick_superiorparietal_medPIQR',
  'vol_inferiorparietal_medPIQR',
  'vol_lateraloccipital_medPIQR',
  'vol_superiortemporal_medPIQR'],
 [],
 [])

In [57]:
# Find the difference between selected features at 0.5 thresold and RF RFECV for left hemisphere
selected_feats_left_rf, selected_feats_left_svm
left_feats_50
common_featsLR, common_featsRR, only_LR_feats, only_RR_feats = find_matchings(left_feats_50,
                                                                             selected_feats_left_rf)
common_featsLR, common_featsRR, only_LR_feats, only_RR_feats

(['area_bankssts_medMIQR'],
 ['area_bankssts_medMIQR'],
 ['thick_caudalanteriorcingulate_medMIQR',
  'thick_isthmuscingulate_medMIQR',
  'thick_medialorbitofrontal_medPIQR',
  'thick_parahippocampal_medPIQR',
  'thick_posteriorcingulate_medMIQR',
  'thick_rostralanteriorcingulate_medPIQR',
  'thick_frontalpole_medMIQR',
  'thick_temporalpole_medMIQR',
  'curv_bankssts_medPIQR',
  'curv_caudalanteriorcingulate_medMIQR',
  'curv_caudalanteriorcingulate_medPIQR',
  'curv_caudalmiddlefrontal_medMIQR',
  'curv_caudalmiddlefrontal_medPIQR',
  'curv_cuneus_medMIQR',
  'curv_cuneus_medPIQR',
  'curv_entorhinal_medMIQR',
  'curv_entorhinal_medPIQR',
  'curv_fusiform_medMIQR',
  'curv_fusiform_medPIQR',
  'curv_inferiorparietal_medMIQR',
  'curv_inferiorparietal_medPIQR',
  'curv_inferiortemporal_medMIQR',
  'curv_inferiortemporal_medPIQR',
  'curv_isthmuscingulate_medMIQR',
  'curv_lateraloccipital_medMIQR',
  'curv_lateraloccipital_medPIQR',
  'curv_lateralorbitofrontal_medPIQR',
  'curv_lingu

In [58]:
# Find the difference between selected features at 0.5 thresold and SVM RFECV for left hemisphere
selected_feats_left_rf, selected_feats_left_svm
left_feats_50
common_featsLR, common_featsRR, only_LR_feats, only_RR_feats = find_matchings(left_feats_50,
                                                                             selected_feats_left_svm)
common_featsLR, common_featsRR, only_LR_feats, only_RR_feats

(['vol_lateraloccipital_medPIQR'],
 ['vol_lateraloccipital_medPIQR'],
 ['thick_caudalanteriorcingulate_medMIQR',
  'thick_isthmuscingulate_medMIQR',
  'thick_medialorbitofrontal_medPIQR',
  'thick_parahippocampal_medPIQR',
  'thick_posteriorcingulate_medMIQR',
  'thick_rostralanteriorcingulate_medPIQR',
  'thick_frontalpole_medMIQR',
  'thick_temporalpole_medMIQR',
  'curv_bankssts_medPIQR',
  'curv_caudalanteriorcingulate_medMIQR',
  'curv_caudalanteriorcingulate_medPIQR',
  'curv_caudalmiddlefrontal_medMIQR',
  'curv_caudalmiddlefrontal_medPIQR',
  'curv_cuneus_medMIQR',
  'curv_cuneus_medPIQR',
  'curv_entorhinal_medMIQR',
  'curv_entorhinal_medPIQR',
  'curv_fusiform_medMIQR',
  'curv_fusiform_medPIQR',
  'curv_inferiorparietal_medMIQR',
  'curv_inferiorparietal_medPIQR',
  'curv_inferiortemporal_medMIQR',
  'curv_inferiortemporal_medPIQR',
  'curv_isthmuscingulate_medMIQR',
  'curv_lateraloccipital_medMIQR',
  'curv_lateraloccipital_medPIQR',
  'curv_lateralorbitofrontal_medPIQR',

In [59]:
# Find the difference between selected features at 0.7 thresold and SVM RFECV for left hemisphere
selected_feats_left_rf, selected_feats_left_svm
left_feats_50
common_featsLR, common_featsRR, only_LR_feats, only_RR_feats = find_matchings(left_feats_70,
                                                                             selected_feats_left_svm)
common_featsLR, common_featsRR, only_LR_feats, only_RR_feats

(['vol_lateraloccipital_medPIQR', 'vol_superiortemporal_medPIQR'],
 ['vol_lateraloccipital_medPIQR', 'vol_superiortemporal_medPIQR'],
 ['thick_caudalanteriorcingulate_medMIQR',
  'thick_cuneus_medMIQR',
  'thick_entorhinal_medMIQR',
  'thick_fusiform_medPIQR',
  'thick_isthmuscingulate_medMIQR',
  'thick_isthmuscingulate_medPIQR',
  'thick_lateraloccipital_medMIQR',
  'thick_lateralorbitofrontal_medMIQR',
  'thick_lingual_medMIQR',
  'thick_medialorbitofrontal_medMIQR',
  'thick_medialorbitofrontal_medPIQR',
  'thick_parahippocampal_medMIQR',
  'thick_parahippocampal_medPIQR',
  'thick_paracentral_medMIQR',
  'thick_parsorbitalis_medMIQR',
  'thick_parsorbitalis_medPIQR',
  'thick_pericalcarine_medMIQR',
  'thick_postcentral_medMIQR',
  'thick_posteriorcingulate_medMIQR',
  'thick_precuneus_medMIQR',
  'thick_rostralanteriorcingulate_medMIQR',
  'thick_rostralanteriorcingulate_medPIQR',
  'thick_frontalpole_medMIQR',
  'thick_temporalpole_medMIQR',
  'thick_transversetemporal_medMIQR',

## Conclusion
RFECV selectes 5 to 7 features. while the most stricted correlation analysis threshold picked 99 features for left and right.<br>
Correlation analysis almost picked the same features for the left and right hemisphere which
indicates that both hemisphere are identical. "Our begining assumption"

# Compare Performance of RFECV & CORR features
## Given:
1. Perforamce of each model using 5 fold CV

## Questions:
1. Which model has the best performance?
2. What are the classifiers that best classify the data?

In [62]:
# Load ML results for each model [ LEFT HEMISPHERE ]
ML_obj_50 = load(os.path.join(results_with_no_FS, subdirs['ML'], 'clf_left_OnlyCorr_50.joblib'))
ML_obj_60 = load(os.path.join(results_with_no_FS, subdirs['ML'], 'clf_left_OnlyCorr_60.joblib'))
ML_obj_70 = load(os.path.join(results_with_no_FS, subdirs['ML'], 'clf_left_OnlyCorr_70.joblib'))
ML_obj_85 = load(os.path.join(results_with_no_FS, subdirs['ML'], 'clf_left_OnlyCorr_85.joblib'))

ML_obj_rf = load(os.path.join(results_with_no_CORRANA, subdirs['ML'], 'clf_left_rf_nocorr.joblib'))
ML_obj_svm = load(os.path.join(results_with_no_CORRANA, subdirs['ML'], 'clf_left_svc_nocorr.joblib'))

In [7]:
def get_results_dict(clc_dict):
    scores_clc = {}
    for clc in clc_dict:
        ML_obj = clc_dict[clc]
        scores_clc[clc] = ML_obj.best_score_
    return scores_clc

In [77]:
classifiers = ['lSVM', 'pagg', 'lg', 'XGB', 'GNB', 'Rf', 'SVC', 'nn']
scores_clc_50 = get_results_dict(ML_obj_50)
scores_clc_60 = get_results_dict(ML_obj_60)
scores_clc_70 = get_results_dict(ML_obj_70)
scores_clc_85 = get_results_dict(ML_obj_85)
scores_clc_rf = get_results_dict(ML_obj_rf)
scores_clc_svm = get_results_dict(ML_obj_svm)

    

In [78]:
scores_clc_50

{'lSVM': 0.5685714285714285,
 'pagg': 0.5856627913149651,
 'lg': 0.5684992302383607,
 'XGB': 0.5684992302383607,
 'GNB': 0.5126389552476509,
 'Rf': 0.5043961352657005,
 'SVC': 0.5672660190051494,
 'nn': 0.5943318999840738}

In [79]:
scores_clc_60

{'lSVM': 0.5725195094760313,
 'pagg': 0.5823289271115358,
 'lg': 0.5684371184371184,
 'XGB': 0.5888373944895684,
 'GNB': 0.5302118171683389,
 'Rf': 0.5022222222222222,
 'SVC': 0.5846817433773956,
 'nn': 0.6068073472421298}

In [81]:
scores_clc_70

{'lSVM': 0.5727021287890853,
 'pagg': 0.5880538302277433,
 'lg': 0.5684376493072145,
 'XGB': 0.5846461750809577,
 'GNB': 0.5269034347295216,
 'Rf': 0.5032850241545894,
 'SVC': 0.6013478791739661,
 'nn': 0.6145256675691458}

In [83]:
scores_clc_85

{'lSVM': 0.5891288421723204,
 'pagg': 0.5944311726920424,
 'lg': 0.594599458512502,
 'XGB': 0.5858438180177311,
 'GNB': 0.5278590009024791,
 'Rf': 0.5021739130434784,
 'SVC': 0.6126384243775549,
 'nn': 0.6187986409725541}

In [84]:
scores_clc_rf

{'lSVM': 0.5458703615225355,
 'pagg': 0.5763316876360356,
 'lg': 0.5524648298561342,
 'XGB': 0.6013239900196422,
 'GNB': 0.5578659022137283,
 'Rf': 0.5141307002176567,
 'SVC': 0.6036200031852206,
 'nn': 0.6093534002229655}

In [85]:
scores_clc_svm

{'lSVM': 0.5558714232627276,
 'pagg': 0.5611753463927377,
 'lg': 0.5547486330095026,
 'XGB': 0.5956728778467909,
 'GNB': 0.5214657323352976,
 'Rf': 0.5306540319583799,
 'SVC': 0.5579396931570845,
 'nn': 0.5942942082072518}

In [88]:
# Load ML results for each model [ Right HEMISPHERE ]
ML_obj_50_r = load(os.path.join(results_with_no_FS, subdirs['ML'], 'clf_right_OnlyCorr_50.joblib'))
ML_obj_60_r = load(os.path.join(results_with_no_FS, subdirs['ML'], 'clf_right_OnlyCorr_60.joblib'))
ML_obj_70_r = load(os.path.join(results_with_no_FS, subdirs['ML'], 'clf_right_OnlyCorr_70.joblib'))
ML_obj_85_r = load(os.path.join(results_with_no_FS, subdirs['ML'], 'clf_right_OnlyCorr_85.joblib'))

ML_obj_rf_r = load(os.path.join(results_with_no_CORRANA, subdirs['ML'], 'clf_right_rf_nocorr.joblib'))
ML_obj_svm_r = load(os.path.join(results_with_no_CORRANA, subdirs['ML'], 'clf_right_svc_nocorr.joblib'))

In [89]:
classifiers = ['lSVM', 'pagg', 'lg', 'XGB', 'GNB', 'Rf', 'SVC', 'nn']
scores_clc_50_r = get_results_dict(ML_obj_50_r)
scores_clc_60_r = get_results_dict(ML_obj_60_r)
scores_clc_70_r = get_results_dict(ML_obj_70_r)
scores_clc_85_r = get_results_dict(ML_obj_85_r)
scores_clc_rf_r = get_results_dict(ML_obj_rf_r)
scores_clc_svm_r = get_results_dict(ML_obj_svm_r)

In [91]:
data_dict = {
    'SVM_left': scores_clc_svm,
    'RF_left': scores_clc_rf,
    '50_left': scores_clc_50,
    '60_left': scores_clc_60,
    '70_left': scores_clc_70,
    '85_left': scores_clc_85,
    'SVM_right': scores_clc_svm_r,
    'RF_right': scores_clc_rf_r,
    '50_right': scores_clc_50_r,
    '60_right': scores_clc_60_r,
    '70_right': scores_clc_70_r,
    '85_right': scores_clc_85_r
}
df = pd.DataFrame(data_dict)
print(df)

      SVM_left   RF_left   50_left   60_left   70_left   85_left  SVM_right  \
lSVM  0.555871  0.545870  0.568571  0.572520  0.572702  0.589129   0.555871   
pagg  0.561175  0.576332  0.585663  0.582329  0.588054  0.594431   0.569174   
lg    0.554749  0.552465  0.568499  0.568437  0.568438  0.594599   0.554749   
XGB   0.595673  0.601324  0.568499  0.588837  0.584646  0.585844   0.596845   
GNB   0.521466  0.557866  0.512639  0.530212  0.526903  0.527859   0.521466   
Rf    0.530654  0.514131  0.504396  0.502222  0.503285  0.502174   0.532828   
SVC   0.557940  0.603620  0.567266  0.584682  0.601348  0.612638   0.557940   
nn    0.594294  0.609353  0.594332  0.606807  0.614526  0.618799   0.584279   

      RF_right  50_right  60_right  70_right  85_right  
lSVM  0.562369  0.568571  0.572520  0.572702  0.589129  
pagg  0.567056  0.564613  0.593319  0.584430  0.595663  
lg    0.584229  0.568499  0.568437  0.568438  0.594599  
XGB   0.622640  0.569597  0.582761  0.586262  0.592857  
GNB

In [95]:
df.max()

SVM_left     0.595673
RF_left      0.609353
50_left      0.594332
60_left      0.606807
70_left      0.614526
85_left      0.618799
SVM_right    0.596845
RF_right     0.629186
50_right     0.598074
60_right     0.605347
70_right     0.616649
85_right     0.618559
dtype: float64

In [96]:
no_nndf = df.drop('nn')

In [97]:
no_nndf.max()

SVM_left     0.595673
RF_left      0.603620
50_left      0.585663
60_left      0.588837
70_left      0.601348
85_left      0.612638
SVM_right    0.596845
RF_right     0.622640
50_right     0.569597
60_right     0.593319
70_right     0.601348
85_right     0.612638
dtype: float64

In [98]:
max_left_scores = [0.595673,
      0.603620,
      0.585663,
     0.588837,
     0.601348]
max_right_scores = [0.596845,
0.622640,
0.569597,
0.593319,
0.601348,
0.612638]

In [99]:
np.max(max_left_scores), np.max(max_right_scores)

(0.60362, 0.62264)

In [100]:
np.mean(max_left_scores), np.mean(max_right_scores)

(0.5950282, 0.5993978333333334)

In [4]:
results_with_no_FS

'./Results'

In [5]:
fs_corr_left_rf = load(os.path.join(results_with_no_FS, subdirs['ML'],
                                    'clf_left_RF_CORR_50.joblib'))
fs_corr_left_svm = load(os.path.join(results_with_no_FS, subdirs['ML'],
                                     'clf_left_lsvm_CORR_50.joblib'))
fs_corr_right_rf = load(os.path.join(results_with_no_FS, subdirs['ML'],
                                     'clf_right_RF_CORR_50.joblib'))
fs_corr_right_svm = load(os.path.join(results_with_no_FS, subdirs['ML'],
                                      'clf_right_lsvm_CORR_50.joblib'))

In [8]:
classifiers = ['lSVM', 'pagg', 'lg', 'XGB', 'GNB', 'Rf', 'SVC', 'nn']
scores_clc_rf_corr_r = get_results_dict(fs_corr_right_rf)
scores_clc_svm_corr_r = get_results_dict(fs_corr_right_svm)
scores_clc_rf_corr_l = get_results_dict(fs_corr_left_rf)
scores_clc_svm_corr_l = get_results_dict(fs_corr_left_svm)

In [10]:
data_dict_FS_CORR = {
    'SVM_corr_left': scores_clc_svm_corr_l,
    'RF_corr_left': scores_clc_rf_corr_l,
    'SVM_corr_right': scores_clc_svm_corr_r,
    'RF_corr_right': scores_clc_rf_corr_r

}
df_corr_FS = pd.DataFrame(data_dict_FS_CORR)
print(df_corr_FS)

      SVM_corr_left  RF_corr_left  SVM_corr_right  RF_corr_right
lSVM       0.581851      0.555500        0.581851       0.577325
pagg       0.525387      0.517711        0.510061       0.512126
lg         0.594835      0.565151        0.594835       0.581673
XGB        0.593980      0.589318        0.592543       0.597011
GNB        0.556755      0.567711        0.556755       0.543484
Rf         0.508817      0.504396        0.508768       0.504396
SVC        0.609934      0.571794        0.609934       0.571902
nn         0.553456      0.563115        0.552225       0.576434


In [11]:
df_corr_FS.max(axis=0)

SVM_corr_left     0.609934
RF_corr_left      0.589318
SVM_corr_right    0.609934
RF_corr_right     0.597011
dtype: float64

In [12]:
nonn_df_corr_fs = df_corr_FS.drop('nn')

In [13]:
nonn_df_corr_fs.max(0)

SVM_corr_left     0.609934
RF_corr_left      0.589318
SVM_corr_right    0.609934
RF_corr_right     0.597011
dtype: float64

In [14]:
nonn_df_corr_fs.mean(0)

SVM_corr_left     0.567366
RF_corr_left      0.553083
SVM_corr_right    0.564964
RF_corr_right     0.555417
dtype: float64

In [15]:
fs_corr_left_rf['nn'].n_features_in_

35

## Conclusion
1. Correlation analysis is a univariate method to find the most relevant features to the target.
2. Recursive feature elimination method is a multivariate method which considers.
3. RFECV selects less features when it is exposed to correlated features than when it is exposed to less correlated features.
4. corr_50 max_acc = (